In [2]:
import pandas as pd
import numpy as np
import regex as re

In [6]:
#Loading dataset into a pandas Dataframe
raw_dataset = pd.read_csv('/content/drive/MyDrive/AI_ML_T_Test/twcs.csv')
raw_dataset.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [7]:
#Dropping columns which are unecessary:
raw_dataset.drop(['author_id', 'created_at'], axis = 1, inplace = True)

In [ ]:
appended_sentences = pd.Series()
for diags in raw_dataset.text:
    diags = diags.replace('.', '</s>, <s>').replace(',', '').replace('@', '').split() #<s> and </s> are tokenizers for start and end of a sentence. These are recognized by tokenizer functions.
    diags.insert(0, '<s>')
    diags.pop(-1)
    pd.concat([appended_sentences, pd.Series(diags)])
raw_dataset.text = appended_sentences

In [ ]:
diag_res_dict = {'Diag':[], 'Res':[]}
for i in raw_dataset.index:
    if raw_dataset['inbound'][i] == True:
        diag_res_dict['Diag'].append(raw_dataset['text'][i])
        diag_res_dict['Res'].append(raw_dataset['text'][raw_dataset.tweet_id == raw_dataset['in_response_to_tweet_id'][i]])
diag_res_dataframe = pd.DataFrame(diag_res_dict)

I have implemented 2 pre-trained models, DialoGPT-medium and Blenderbot by Facebook. I have initially used pre-trained weights and biases which have been derived from training on popular datasets such as Blended_skill_talk, Wiki_corpus and such.

I have then try to fine tune the models based on the requirements of the chatbot with the given dataset.

In [ ]:
! pip install flask transformers torch

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, Conversation, ConversationalPipeline
import torch

In [ ]:
#Loading the blenderbot tokenizer and the pre-trained blenderbot model
checkpoint_blender = 'facebook/blenderbot-400M-distill'
tokenizer_blender = AutoTokenizer.from_pretrained(checkpoint_blender)
model_blender = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_blender)

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids_blender = tokenizer_blender.encode(input(">> User:") + tokenizer_blender.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids_blender = torch.cat([chat_history_ids_blender, new_user_input_ids_blender], dim=-1) if step > 0 else new_user_input_ids_blender
     
    chat_history_ids_blender = model_blender.generate(bot_input_ids_blender, max_length=1000, pad_token_id=tokenizer_blender.eos_token_id)

    print(tokenizer_blender.decode(chat_history_ids_blender[0], skip_special_tokens = True))

In [ ]:
tokenizer_dialo = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large")
model_dialo = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-large")

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer_dialo.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
     
    chat_history_ids = model_dialo.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    print("DialoGPT: {}".format(tokenizer_dialo.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))
    